In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers
!pip install sentence_transformers
!pip install pyarabic
!pip install langdetect

     |████████████████████████████████| 1.9MB 7.6MB/s 
     |████████████████████████████████| 3.2MB 17.5MB/s 
     |████████████████████████████████| 890kB 41.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=86d02e2c5e71767a9e51ee77713a53edf5fcf62a8697b2fbd9b91c3276aa9ed3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 71kB 4.6MB/s 
     |████████████████████████████████| 1.2MB 10.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp37-none-any.whl size=103068 sha256=4cc4d94c6dd30e1ad3d85b69618a920a8b35662cb7095a17a43728e74927a1c1
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
Successfully built sentence-transformers
     |████████████████████████████████| 112kB 7.5MB/s 
  Created wheel for pyarabic: filename

In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses, util, SentencesDataset
from torch.utils.data import DataLoader

In [5]:
# !pkill "java"
# gateway = JavaGateway.launch_gateway(classpath='./FarasaSegmenterJar.jar')
# farasa_segmenter = gateway.jvm.com.qcri.farasa.segmenter.Farasa()

#farasa_segmenter = FarasaSegmenter(interactive=True)
#df_AJGT = pd.read_excel('drive/MyDrive/gebriel_cleaned.xlsx',header=0)
df_AJGT = pd.read_csv('drive/MyDrive/mtn_data_labeled.csv',header=0)
df_AJGT = df_AJGT.iloc[1:]

DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

df_AJGT = df_AJGT[['text', 'label']]
df_AJGT.columns = [DATA_COLUMN, LABEL_COLUMN]

label_map = {
    'data_offer' : 0,
    'voice_offers' : 1,
    'complaint' : 2,
    'service_center' : 3,
    'service_centers' : 3,
    'my_sim' : 4
}

df_AJGT = df_AJGT.dropna()

#df_AJGT[DATA_COLUMN] = df_AJGT[DATA_COLUMN].apply(lambda x: preprocess(x, do_farasa_tokenization=True , farasa=farasa_segmenter, use_farasapy = True))
df_AJGT[LABEL_COLUMN] = df_AJGT[LABEL_COLUMN].apply(lambda x: label_map[x])


#train_AJGT, test_AJGT = train_test_split(df_AJGT, test_size=0.2,random_state=42)

In [6]:
%load_ext google.colab.data_table

In [7]:
df_AJGT

,text,label
1,شهر إنترنت كيف,0
2,انا عندي شريحا اريبا داير اغلبه داتا ممكن,1
3,ماهى العروض الجديدة ؟,0
4,بسال من جهاز wifi 4G منشط لمدة سنة بكلف كم,0
5,انا داير اعرف كيف انشط بالشهر,0
...,...,...
766,عاوزه اعرف رقم PUK,4
767,كيف معرفه رقم puk,4
768,كيف انشط النت,0
769,كيف اعرف رمز puk,4


In [8]:
training_data = []
for index, row in df_AJGT.iterrows():
  training_data.append(InputExample(texts=[row['text']], label=row['label']))

In [9]:
!wget https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/bert-base-nli-stsb-mean-tokens.zip

--2021-03-04 13:43:10--  https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/bert-base-nli-stsb-mean-tokens.zip
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405233603 (386M) [application/zip]
Saving to: ‘bert-base-nli-stsb-mean-tokens.zip’

bert-base-nli-stsb- 100%[===================>] 386.46M  20.6MB/s    in 23s     

2021-03-04 13:43:33 (17.1 MB/s) - ‘bert-base-nli-stsb-mean-tokens.zip’ saved [405233603/405233603]



In [10]:
import zipfile
with zipfile.ZipFile("bert-base-nli-stsb-mean-tokens.zip", 'r') as zip_ref:
    zip_ref.extractall("Suda")

In [11]:
import shutil
shutil.copy("drive/MyDrive/new_sudaBERT/vocab.txt","Suda/0_BERT/")
shutil.copy("drive/MyDrive/new_sudaBERT/config.json","Suda/0_BERT/")
shutil.copy("drive/MyDrive/new_sudaBERT/pytorch_model.bin","Suda/0_BERT/")

'Suda/0_BERT/pytorch_model.bin'

In [12]:


#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('Suda')

#Define your train examples. You need more than just two examples...
train_examples = training_data

#Define your train dataset, the dataloader and the train loss
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
train_loss = losses.BatchSemiHardTripletLoss(model=model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3, warmup_steps=100)

In [ ]:
list(df_AJGT.loc[df_AJGT['label'] == 0]['text'])

[' شهر إنترنت كيف ',
 ' ماهى العروض الجديدة ؟ ',
 ' بسال من جهاز wifi 4G  منشط لمدة سنة بكلف كم ',
 ' انا داير اعرف كيف انشط بالشهر ',
 ' كيف اشترك في حزمة 10 G في ال4G ',
 ' ممكن اسأل عن سرعة الانترنت الذهبي دا كم؟؟؟   ',
 ' طييييب ي ادمن  انا عاوز لي جهاز واي فاي منكم  ممكن توريني العروض والاسعار والسرعة  واعتذر جدا ع الاطالة  ',
 ' ديل يعني م محدودات😳😳    ؟؟؟؟؟؟؟ ',
 ' ممكن توروني كااافة العروض بتاعة شريحة البيانات واسعارها ',
 ' والعرض بتاع ساعتين غير محدودات خلال اليوم ',
 ' لو عاوز اعرف متبقي لي كم من الانترنت اليومي ل شريحة البيانات الخط اعمل شنو؟؟ ',
 ' بسإل عن خدمات الشهر  Mujahid Aljili:   الجيل الرابع ',
 ' ممكن سعر شريحة البيانات ',
 ' لو داير انشط لامحدود البلاتيني اضغط * كم؟ ',
 ' عاوز انشط باقت الانتر نت الزهبي لي شهر',
 ' داير انشط الباقة الشهرية الزهبية',
 ' كيف ممكن اعرف كم فضل لي في الباقة',
 ' بس لو سمحت حجم الباقة اليومي كم ميجا  ',
 '  ممكن اعرف لو خدمات اللامحدود الشهرية ممكن تتفعل في الشريحة العادية ولا لا',
 ' عندي شريحة داتا ام تي إن ونشطت فيها خمسة قيقا بس بت

In [13]:
class1 = list(df_AJGT.loc[df_AJGT['label'] == 0]['text'])
class2 = list(df_AJGT.loc[df_AJGT['label'] == 1]['text'])
class3 = list(df_AJGT.loc[df_AJGT['label'] == 2]['text'])
class4 = list(df_AJGT.loc[df_AJGT['label'] == 3]['text'])
class5 = list(df_AJGT.loc[df_AJGT['label'] == 4]['text'])

v = {}
v[0] = class1
v[1] = class2
v[2] = class3
v[3] = class4
v[4] = class5

classes = v[0]+v[1]+v[2]+v[3]+v[4]

#Compute embeddings
embeddings2 = model.encode(classes, convert_to_tensor=True)

In [14]:
sentences = ['داير اعرف عروض المكالمات العندكم']
embeddings = model.encode(sentences, convert_to_tensor=True)

In [15]:
#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings2)
euclidena_dist = len(embeddings)*[len(embeddings2)*[0]]
for i in range(len(embeddings)):
  for j in range(len(embeddings2)):
    euclidena_dist[i][j] = sum(((embeddings[i] - embeddings2[j])**2).reshape(768))


#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)):
    for j in range(len(cosine_scores[0])):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j], 'score2': euclidena_dist[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score2'])

for pair in pairs[:10]:
    i, j = pair['index']
    print("{}\t \t{}\t Score: {:.4f}\t Score2: {:.4f}".format(sentences[i], classes[j], pair['score'], pair['score2']), end='\t')
    for k in range(len(v)):
      if classes[j] in v[k]:
        break
    print(k)

داير اعرف عروض المكالمات العندكم	 	    عاوز اعرف عروض خدمة المكالمات والاسعار   	 Score: 0.9802	 Score2: 15.7182	1
داير اعرف عروض المكالمات العندكم	 	 عايزه اعرف طريقه تحويل الرصيد ...وخدمات المكالمات بتتعمل كيف 	 Score: 0.9709	 Score2: 23.0850	1
داير اعرف عروض المكالمات العندكم	 	 خدمه تحويل المكالمات 	 Score: 0.9555	 Score2: 34.8586	4
داير اعرف عروض المكالمات العندكم	 	  عايذه اعرف رموز خدمة المكالمات 	 Score: 0.9286	 Score2: 56.4162	1
داير اعرف عروض المكالمات العندكم	 	 ممكن اعرف جميع خدمات مكالمات MTN وطريقه الاشتراك 	 Score: 0.9189	 Score2: 64.3347	1
داير اعرف عروض المكالمات العندكم	 	 هل تدعم شريحة البيانات المكالمات داخل وخارج الشبكة   	 Score: 0.9128	 Score2: 68.7903	4
داير اعرف عروض المكالمات العندكم	 	 طريقه تحويل الرصيد	 Score: 0.9028	 Score2: 75.9635	4
داير اعرف عروض المكالمات العندكم	 	 طريقه تحويل الرصيد	 Score: 0.9028	 Score2: 75.9635	4
داير اعرف عروض المكالمات العندكم	 	 طريقه التحويل	 Score: 0.9000	 Score2: 77.4986	4
داير اعرف عروض المكالمات العندكم	 	 هل يمكن تحويل شر